In [2]:
:dep futures = {version = "0.3"}

use futures::join;
use futures::try_join;

In [18]:
#[derive(Debug)]
struct Book;

#[derive(Debug)]
struct Music;

async fn get_book() -> Book{
    
    Ok(Book)
}

async fn get_music() -> Music{
    Ok(Music)
}

async fn get_book_and_music() -> (Book, Music){
    let book_fut = get_book();
    let music_fut = get_music();
    join!(book_fut, music_fut)
    
}

Error: type mismatch resolving `<impl core::future::future::Future as core::future::future::Future>::Output == Music`

Error: type mismatch resolving `<impl core::future::future::Future as core::future::future::Future>::Output == Book`

In [17]:
println!("{:?}", get_book_and_music())

Error: `impl core::future::future::Future` doesn't implement `std::fmt::Debug`